In [84]:
from get_data import request_page
import string
import pandas as pd
b_url = 'https://www.basketball-reference.com'
players_url = 'https://www.basketball-reference.com/players/'

## Get data and url for all players

In [22]:
alphabet = list(string.ascii_lowercase)
list_urls = [players_url + letter + '/' for letter in alphabet] 
list_urls[:2]

['https://www.basketball-reference.com/players/a/',
 'https://www.basketball-reference.com/players/b/']

### Try with players starting with a

In [23]:
test = list_urls[0]
test

'https://www.basketball-reference.com/players/a/'

In [24]:
players_soup = request_page(test)

In [69]:
cols = [elem.text.lower() for elem in players_soup.find('thead').find_all('th')]
cols.insert(0, "url")
cols

['url', 'player', 'from', 'to', 'pos', 'ht', 'wt', 'birth date', 'colleges']

In [75]:
player1 = players_soup.find('tbody').find_all('tr')[0]
player1

<tr><th class="left" data-append-csv="abdelal01" data-stat="player" scope="row"><a href="/players/a/abdelal01.html">Alaa Abdelnaby</a></th><td class="right" data-stat="year_min">1991</td><td class="right" data-stat="year_max">1995</td><td class="center" data-stat="pos">F-C</td><td class="right" csk="82.0" data-stat="height">6-10</td><td class="right" data-stat="weight">240</td><td class="left" csk="19680624" data-stat="birth_date"><a href="/friv/birthdays.cgi?month=6&amp;day=24">June 24, 1968</a></td><td class="left" data-stat="colleges"><a href="/friv/colleges.fcgi?college=duke">Duke</a></td></tr>

In [71]:
# URL
b_url + player1.a['href']

'https://www.basketball-reference.com/players/a/abdelal01.html'

In [59]:
# name
player1.a.text

'Alaa Abdelnaby'

In [66]:
# rest of columns
[elem.text for elem in player1.find_all('td')]

['1991', '1995', 'F-C', '6-10', '240', 'June 24, 1968', 'Duke']

In [87]:
def extract_player_info(player_text, b_url = b_url):
    url = b_url + player_text.a['href']
    name = player_text.a.text
    info = [name, url]
    info.extend([elem.text for elem in player_text.find_all('td')])
    return info

In [88]:
a_players = players_soup.find('tbody').find_all('tr')
result = [extract_player_info(player_text) for player_text in a_players]
pd.DataFrame(result, columns = cols)

### Creating a function

----

That creates a dataframe from a url

In [92]:
def df_from_url(url):
    players_soup = request_page(url)
    
    cols = [elem.text.lower() for elem in players_soup.find('thead').find_all('th')]
    cols.insert(0, "url")
    
    
    a_players = players_soup.find('tbody').find_all('tr')
    result = [extract_player_info(player_text) for player_text in a_players]
    df = pd.DataFrame(result, columns = cols)
    
#     print(f'Created dataframe with information of {df.shape[0]} players.')
    return df


In [96]:
df0 = df_from_url(list_urls[0])
df1 = df_from_url(list_urls[1])

df = pd.concat([df0,df1], axis = 0)
df.shape

(625, 9)

### Create a function that performs this steps and returns a complete dataframe for all players

In [110]:
def get_players_data(players_url='https://www.basketball-reference.com/players/'):
    
    alphabet = list(string.ascii_lowercase)
    list_urls = [players_url + letter + '/' for letter in alphabet]
    df0 = df_from_url(list_urls[0])
    
    df_list = [df_from_url(url) for url in list_urls]
    df = pd.concat(df_list, axis = 0)
    print(df.shape)
    return df
        

In [111]:
dff = get_players_data()

(4800, 9)


In [112]:
dff

,url,player,from,to,pos,ht,wt,birth date,colleges
0,Alaa Abdelnaby,https://www.basketball-reference.com/players/a...,1991,1995,F-C,6-10,240,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,https://www.basketball-reference.com/players/a...,1969,1978,C-F,6-9,235,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar,https://www.basketball-reference.com/players/a...,1970,1989,C,7-2,225,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,https://www.basketball-reference.com/players/a...,1991,2001,G,6-1,162,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,https://www.basketball-reference.com/players/a...,1998,2003,F,6-6,223,"November 3, 1974","Michigan, San Jose State"
...,...,...,...,...,...,...,...,...,...
15,Ante Žižić,https://www.basketball-reference.com/players/z...,2018,2020,F-C,6-10,254,"January 4, 1997",
16,Jim Zoet,https://www.basketball-reference.com/players/z...,1983,1983,C,7-1,240,"December 20, 1953",Kent State University
17,Bill Zopf,https://www.basketball-reference.com/players/z...,1971,1971,G,6-1,170,"June 7, 1948",Duquesne
18,Ivica Zubac,https://www.basketball-reference.com/players/z...,2017,2020,C,7-0,240,"March 18, 1997",


## Enter players pages